Used a naive RNN cell and then a lstm cell to predict the minimum and maximum temperature using a given data of weather report during the World War 2. The data for this project was taken from Kaggle.

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import time

In [2]:
def rnn_step(x, h_prev, Wxh, bh, Whh, Why, by, activation=tf.nn.relu):
    
    hidden = tf.matmul(x, Wxh)
    state = tf.matmul(h_prev, Whh)
    h = tf.nn.tanh(hidden + state + bh)
    out = activation(tf.matmul(h, Why) + by)
    return h, out

In [3]:
def rnn_forward(x, h_ini, Wxh, bh, Whh, Why, by, activation=tf.nn.leaky_relu):
    
    if x.ndim is not 3:
        x = tf.expand_dims(x, 0)
    
    t = x.shape[1]
    h = np.copy(h_ini)
    for i in range(t):
        xs = x[:, i, :]
        if xs.ndim is not 2:
            xs = tf.expand_dims(xs, 0)
        h, out = rnn_step(xs, h, Wxh, bh, Whh, Why, by, activation)
    return out

In [2]:
def seperate_labels(x, window, step):
    
    assert window > step
    
    final_x = x[0:window-1]
    final_x = tf.expand_dims(final_x, axis=0)
    
    final_y = x[window]
    final_y = tf.expand_dims(final_y, axis=0)
    
    for i in range(step, len(x)-window, step):
        xt = x[i:i+window-1]
        xt = tf.expand_dims(xt, axis=0)
        final_x = tf.concat([final_x, xt], axis=0)
        
        yt = x[i+window]
        yt = tf.expand_dims(yt, axis=0)
        final_y = tf.concat([final_y, yt], axis=0)
        
    return final_x, final_y

In [3]:
def normalize(data, mean=None, std=None):
    if mean==None:
        mean = tf.reduce_mean(data, axis=0)
    if std==None:
        std = tf.reduce_max(data, axis=0)-tf.reduce_min(data, axis=0)
    normalized = (data - mean)/std + 1
    return normalized, mean, std

In [4]:
def inverse_transform(data, mean, std):
    out = (data-1) * std + mean
    return out

In [5]:
data = pd.read_csv('./data/weatherww2/Summary of Weather.csv', usecols=['MaxTemp', 'MinTemp']).values
stations = pd.read_csv('./data/weatherww2/Summary of Weather.csv', usecols=['STA']).values
stations= stations.reshape(-1)
idx = tf.where(stations==10001)
idx = tf.reshape(idx, [-1])
data = data[idx]
print(data.shape)

(1157, 2)


In [6]:
train_data = data[0:800]
cv_data = data[800:1000]
test_data = data[1000:1150]

train_data, train_mean, train_std = normalize(train_data)
cv_data,_,_ = normalize(cv_data, train_mean, train_std)

X, Y = seperate_labels(train_data, window=4, step=1)
x_cv, y_cv = seperate_labels(cv_data, 4, 1)

print(X.shape, Y.shape)

(796, 3, 2) (796, 2)


In [7]:
num_samples, time_step, num_features = X.shape
hidden_size = 2
out_size = Y.shape[-1]
learning_rate = 0.001
epochs = 1000
mini_batch = 10

In [14]:
h_ini = [[0.,0.]]

Wxh = tf.Variable(np.random.randn(num_features, hidden_size)*0.1)
Whh = tf.Variable(np.random.randn(hidden_size, hidden_size)*0.1)
bh = tf.Variable(np.random.randn(hidden_size)*0.1)

Why = tf.Variable(np.random.randn(hidden_size, out_size)*0.1)
by = tf.Variable(np.random.randn(out_size)*0.1)

optimizer = tf.keras.optimizers.Adam(learning_rate)
variables = [Wxh, Whh, bh, Why, by]

print(Wxh, Whh)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float64, numpy=
array([[-0.10927166,  0.02234396],
       [ 0.02736561,  0.0582876 ]])> <tf.Variable 'Variable:0' shape=(2, 2) dtype=float64, numpy=
array([[-0.09989175,  0.0429998 ],
       [-0.06979307, -0.17064172]])>


In [15]:
X, Y = seperate_labels(train_data, window=4, step=1)
x_cv, y_cv = seperate_labels(cv_data, 4, 1)
learning_rate = 0.001
mini_batch = 10
for i in range(epochs):
    print("EPOCH", i)
    start = time.time()
    total_loss = 0.
    for j in range(0, num_samples-mini_batch, mini_batch):
        xs = X[j:j+mini_batch,:,:]
        ys = Y[j:j+mini_batch]
        with tf.GradientTape(persistent=True, watch_accessed_variables=False) as t:
            t.watch([Wxh, Whh, bh, Why, by])
            out = rnn_forward(xs, h_ini, Wxh, bh, Whh, Why, by)
            loss = tf.reduce_sum(tf.math.square(out-ys))/mini_batch
            total_loss += loss
        Wxh.assign_sub(learning_rate*t.gradient(loss,Wxh))
        Whh.assign_sub(learning_rate*t.gradient(loss,Whh))
        bh.assign_sub(learning_rate*t.gradient(loss,bh))
        Why.assign_sub(learning_rate*t.gradient(loss,Why))
        by.assign_sub(learning_rate*t.gradient(loss,by))
        del t
    end = time.time()
    mean_loss = total_loss/(num_samples//mini_batch)
    print("loss at end of EPOCH: %0.4f" % mean_loss)
    print("time taken: {:0.2f}s".format(end-start))
    y_predict = rnn_forward(x_cv, h_ini, Wxh, bh, Whh, Why, by)
    cv_loss = tf.reduce_sum(tf.math.square(y_predict-y_cv))/len(x_cv)
    print("validation loss:", cv_loss.numpy())
    print("Done with EPOCH")

EPOCH 0
loss at end of EPOCH: 1.2217
time taken: 1.31s
validation loss: 1.390253390270983
Done with EPOCH
EPOCH 1
loss at end of EPOCH: 0.8428
time taken: 1.30s
validation loss: 0.9951154483460689
Done with EPOCH
EPOCH 2
loss at end of EPOCH: 0.5677
time taken: 1.31s
validation loss: 0.7003188186683963
Done with EPOCH
EPOCH 3
loss at end of EPOCH: 0.3736
time taken: 1.33s
validation loss: 0.48656407636532645
Done with EPOCH
EPOCH 4
loss at end of EPOCH: 0.2423
time taken: 1.28s
validation loss: 0.33702246996332624
Done with EPOCH
EPOCH 5
loss at end of EPOCH: 0.1579
time taken: 1.31s
validation loss: 0.23593957485637168
Done with EPOCH
EPOCH 6
loss at end of EPOCH: 0.1062
time taken: 1.31s
validation loss: 0.1693954228778298
Done with EPOCH
EPOCH 7
loss at end of EPOCH: 0.0759
time taken: 1.29s
validation loss: 0.12626447669151597
Done with EPOCH
EPOCH 8
loss at end of EPOCH: 0.0588
time taken: 1.27s
validation loss: 0.09844335778750735
Done with EPOCH
EPOCH 9
loss at end of EPOCH: 0.0

loss at end of EPOCH: 0.0379
time taken: 1.26s
validation loss: 0.042223969074924865
Done with EPOCH
EPOCH 77
loss at end of EPOCH: 0.0378
time taken: 1.25s
validation loss: 0.04219428825513882
Done with EPOCH
EPOCH 78
loss at end of EPOCH: 0.0378
time taken: 1.27s
validation loss: 0.042164578791692435
Done with EPOCH
EPOCH 79
loss at end of EPOCH: 0.0378
time taken: 1.29s
validation loss: 0.042134840692228843
Done with EPOCH
EPOCH 80
loss at end of EPOCH: 0.0378
time taken: 1.30s
validation loss: 0.04210507396515591
Done with EPOCH
EPOCH 81
loss at end of EPOCH: 0.0378
time taken: 1.28s
validation loss: 0.04207527861964996
Done with EPOCH
EPOCH 82
loss at end of EPOCH: 0.0377
time taken: 1.28s
validation loss: 0.04204545466565739
Done with EPOCH
EPOCH 83
loss at end of EPOCH: 0.0377
time taken: 1.29s
validation loss: 0.04201560211389808
Done with EPOCH
EPOCH 84
loss at end of EPOCH: 0.0377
time taken: 1.27s
validation loss: 0.041985720975867856
Done with EPOCH
EPOCH 85
loss at end of 

loss at end of EPOCH: 0.0363
time taken: 1.30s
validation loss: 0.0399198951110183
Done with EPOCH
EPOCH 152
loss at end of EPOCH: 0.0362
time taken: 1.32s
validation loss: 0.03988814068069515
Done with EPOCH
EPOCH 153
loss at end of EPOCH: 0.0362
time taken: 1.28s
validation loss: 0.039856360392335505
Done with EPOCH
EPOCH 154
loss at end of EPOCH: 0.0362
time taken: 1.28s
validation loss: 0.03982455431410227
Done with EPOCH
EPOCH 155
loss at end of EPOCH: 0.0362
time taken: 1.27s
validation loss: 0.03979272251498744
Done with EPOCH
EPOCH 156
loss at end of EPOCH: 0.0362
time taken: 1.28s
validation loss: 0.03976086506481294
Done with EPOCH
EPOCH 157
loss at end of EPOCH: 0.0361
time taken: 1.28s
validation loss: 0.039728982034230254
Done with EPOCH
EPOCH 158
loss at end of EPOCH: 0.0361
time taken: 1.29s
validation loss: 0.03969707349472087
Done with EPOCH
EPOCH 159
loss at end of EPOCH: 0.0361
time taken: 1.31s
validation loss: 0.039665139518596446
Done with EPOCH
EPOCH 160
loss at 

loss at end of EPOCH: 0.0346
time taken: 1.31s
validation loss: 0.03747216833283743
Done with EPOCH
EPOCH 227
loss at end of EPOCH: 0.0345
time taken: 1.34s
validation loss: 0.037438719705308474
Done with EPOCH
EPOCH 228
loss at end of EPOCH: 0.0345
time taken: 1.38s
validation loss: 0.037405252628685215
Done with EPOCH
EPOCH 229
loss at end of EPOCH: 0.0345
time taken: 1.32s
validation loss: 0.037371767232793854
Done with EPOCH
EPOCH 230
loss at end of EPOCH: 0.0345
time taken: 1.40s
validation loss: 0.03733826364825551
Done with EPOCH
EPOCH 231
loss at end of EPOCH: 0.0345
time taken: 1.39s
validation loss: 0.03730474200648496
Done with EPOCH
EPOCH 232
loss at end of EPOCH: 0.0344
time taken: 1.38s
validation loss: 0.03727120243969003
Done with EPOCH
EPOCH 233
loss at end of EPOCH: 0.0344
time taken: 1.38s
validation loss: 0.037237645080869235
Done with EPOCH
EPOCH 234
loss at end of EPOCH: 0.0344
time taken: 1.39s
validation loss: 0.03720407006381067
Done with EPOCH
EPOCH 235
loss a

loss at end of EPOCH: 0.0328
time taken: 1.28s
validation loss: 0.03492203004542888
Done with EPOCH
EPOCH 302
loss at end of EPOCH: 0.0328
time taken: 1.27s
validation loss: 0.03488760964283943
Done with EPOCH
EPOCH 303
loss at end of EPOCH: 0.0328
time taken: 1.27s
validation loss: 0.03485318261351521
Done with EPOCH
EPOCH 304
loss at end of EPOCH: 0.0328
time taken: 1.32s
validation loss: 0.03481874914144726
Done with EPOCH
EPOCH 305
loss at end of EPOCH: 0.0327
time taken: 1.30s
validation loss: 0.03478430941124762
Done with EPOCH
EPOCH 306
loss at end of EPOCH: 0.0327
time taken: 1.29s
validation loss: 0.03474986360814435
Done with EPOCH
EPOCH 307
loss at end of EPOCH: 0.0327
time taken: 1.26s
validation loss: 0.03471541191797903
Done with EPOCH
EPOCH 308
loss at end of EPOCH: 0.0327
time taken: 1.29s
validation loss: 0.03468095452720308
Done with EPOCH
EPOCH 309
loss at end of EPOCH: 0.0326
time taken: 1.27s
validation loss: 0.03464649162287369
Done with EPOCH
EPOCH 310
loss at en

...

loss at end of EPOCH: 0.0255
time taken: 1.28s
validation loss: 0.023358916590057146
Done with EPOCH
EPOCH 677
loss at end of EPOCH: 0.0255
time taken: 1.28s
validation loss: 0.023336170169834723
Done with EPOCH
EPOCH 678
loss at end of EPOCH: 0.0255
time taken: 1.26s
validation loss: 0.02331348358161886
Done with EPOCH
EPOCH 679
loss at end of EPOCH: 0.0254
time taken: 1.28s
validation loss: 0.023290856868664712
Done with EPOCH
EPOCH 680
loss at end of EPOCH: 0.0254
time taken: 1.32s
validation loss: 0.023268290073066394
Done with EPOCH
EPOCH 681
loss at end of EPOCH: 0.0254
time taken: 1.28s
validation loss: 0.023245783235756785
Done with EPOCH
EPOCH 682
loss at end of EPOCH: 0.0254
time taken: 1.30s
validation loss: 0.02322333639650787
Done with EPOCH
EPOCH 683
loss at end of EPOCH: 0.0254
time taken: 1.29s
validation loss: 0.02320094959393062
Done with EPOCH
EPOCH 684
loss at end of EPOCH: 0.0254
time taken: 1.27s
validation loss: 0.023178622865475354
Done with EPOCH
EPOCH 685
loss

loss at end of EPOCH: 0.0247
time taken: 1.30s
validation loss: 0.021820487833090808
Done with EPOCH
EPOCH 752
loss at end of EPOCH: 0.0246
time taken: 1.32s
validation loss: 0.02180227125224694
Done with EPOCH
EPOCH 753
loss at end of EPOCH: 0.0246
time taken: 1.29s
validation loss: 0.021784114582731096
Done with EPOCH
EPOCH 754
loss at end of EPOCH: 0.0246
time taken: 1.29s
validation loss: 0.021766017783907545
Done with EPOCH
EPOCH 755
loss at end of EPOCH: 0.0246
time taken: 1.29s
validation loss: 0.02174798081411085
Done with EPOCH
EPOCH 756
loss at end of EPOCH: 0.0246
time taken: 1.29s
validation loss: 0.021730003630649287
Done with EPOCH
EPOCH 757
loss at end of EPOCH: 0.0246
time taken: 1.27s
validation loss: 0.021712086189808178
Done with EPOCH
EPOCH 758
loss at end of EPOCH: 0.0246
time taken: 1.29s
validation loss: 0.021694228446853852
Done with EPOCH
EPOCH 759
loss at end of EPOCH: 0.0246
time taken: 1.26s
validation loss: 0.021676430356037148
Done with EPOCH
EPOCH 760
los

loss at end of EPOCH: 0.0241
time taken: 1.31s
validation loss: 0.020616571488492556
Done with EPOCH
EPOCH 827
loss at end of EPOCH: 0.0240
time taken: 1.29s
validation loss: 0.020602670925252812
Done with EPOCH
EPOCH 828
loss at end of EPOCH: 0.0240
time taken: 1.27s
validation loss: 0.020588824653640807
Done with EPOCH
EPOCH 829
loss at end of EPOCH: 0.0240
time taken: 1.26s
validation loss: 0.02057503256815464
Done with EPOCH
EPOCH 830
loss at end of EPOCH: 0.0240
time taken: 1.27s
validation loss: 0.020561294562628087
Done with EPOCH
EPOCH 831
loss at end of EPOCH: 0.0240
time taken: 1.30s
validation loss: 0.02054761053023701
Done with EPOCH
EPOCH 832
loss at end of EPOCH: 0.0240
time taken: 1.27s
validation loss: 0.020533980363505136
Done with EPOCH
EPOCH 833
loss at end of EPOCH: 0.0240
time taken: 1.27s
validation loss: 0.020520403954310047
Done with EPOCH
EPOCH 834
loss at end of EPOCH: 0.0240
time taken: 1.28s
validation loss: 0.02050688119388947
Done with EPOCH
EPOCH 835
loss

loss at end of EPOCH: 0.0236
time taken: 1.26s
validation loss: 0.019716862888078987
Done with EPOCH
EPOCH 902
loss at end of EPOCH: 0.0236
time taken: 1.27s
validation loss: 0.019706704044516105
Done with EPOCH
EPOCH 903
loss at end of EPOCH: 0.0236
time taken: 1.26s
validation loss: 0.019696590153002517
Done with EPOCH
EPOCH 904
loss at end of EPOCH: 0.0236
time taken: 1.28s
validation loss: 0.019686521075386692
Done with EPOCH
EPOCH 905
loss at end of EPOCH: 0.0236
time taken: 1.26s
validation loss: 0.019676496673316838
Done with EPOCH
EPOCH 906
loss at end of EPOCH: 0.0236
time taken: 1.27s
validation loss: 0.019666516808246215
Done with EPOCH
EPOCH 907
loss at end of EPOCH: 0.0236
time taken: 1.26s
validation loss: 0.01965658134144002
Done with EPOCH
EPOCH 908
loss at end of EPOCH: 0.0236
time taken: 1.27s
validation loss: 0.019646690133980516
Done with EPOCH
EPOCH 909
loss at end of EPOCH: 0.0236
time taken: 1.26s
validation loss: 0.01963684304677339
Done with EPOCH
EPOCH 910
los

loss at end of EPOCH: 0.0233
time taken: 1.27s
validation loss: 0.01907021750557479
Done with EPOCH
EPOCH 977
loss at end of EPOCH: 0.0233
time taken: 1.28s
validation loss: 0.019063040001762514
Done with EPOCH
EPOCH 978
loss at end of EPOCH: 0.0233
time taken: 1.26s
validation loss: 0.01905589690570333
Done with EPOCH
EPOCH 979
loss at end of EPOCH: 0.0233
time taken: 1.29s
validation loss: 0.019048788079064568
Done with EPOCH
EPOCH 980
loss at end of EPOCH: 0.0233
time taken: 1.28s
validation loss: 0.01904171338368606
Done with EPOCH
EPOCH 981
loss at end of EPOCH: 0.0233
time taken: 1.28s
validation loss: 0.01903467268158409
Done with EPOCH
EPOCH 982
loss at end of EPOCH: 0.0233
time taken: 1.29s
validation loss: 0.019027665834955274
Done with EPOCH
EPOCH 983
loss at end of EPOCH: 0.0233
time taken: 1.30s
validation loss: 0.01902069270618002
Done with EPOCH
EPOCH 984
loss at end of EPOCH: 0.0233
time taken: 1.29s
validation loss: 0.019013753157826083
Done with EPOCH
EPOCH 985
loss a

In [19]:
test_data = data[1000:1150]
test_data,_,_ = normalize(test_data, train_mean, train_std)
x_test, y_test = seperate_labels(test_data, 4, 1)
print(x_test.shape, y_test.shape)
y_predict = rnn_forward(x_test, h_ini, Wxh, bh, Whh, Why, by)

print(y_predict.shape)
y_test = inverse_transform(y_test, train_mean, train_std)
y_predict = inverse_transform(y_predict, train_mean, train_std)
final = np.hstack([y_test, y_predict])
print(final)
loss = tf.math.sqrt(tf.reduce_sum(tf.math.square(y_predict-y_test))/len(x_test))
print("Test error: ", loss.numpy())

(146, 3, 2) (146, 2)
(146, 2)
[[32.22222222 23.33333333 32.7182184  23.40876544]
 [35.         23.88888889 32.97545426 23.59371282]
 [33.33333333 23.88888889 31.96394459 23.05897152]
 [33.88888889 23.33333333 33.17039202 23.63791889]
 [34.44444444 25.         32.60018656 23.36744522]
 [34.44444444 25.55555556 32.6645815  23.38822343]
 [33.88888889 25.55555556 33.23294149 23.68606854]
 [33.33333333 27.77777778 33.41618068 23.77445464]
 [33.33333333 26.66666667 33.2067619  23.67764296]
 [34.44444444 23.88888889 33.35966076 23.79629051]
 [33.88888889 24.44444444 33.18268379 23.68660289]
 [33.33333333 25.         33.09808355 23.59068029]
 [34.44444444 25.         32.94344183 23.5389305 ]
 [32.77777778 23.88888889 32.79477614 23.48254988]
 [33.88888889 24.44444444 33.25735744 23.69288269]
 [33.88888889 25.55555556 32.36579344 23.25799119]
 [30.         22.77777778 32.85299804 23.50155729]
 [32.77777778 22.22222222 33.13107453 23.64812822]
 [33.88888889 23.88888889 30.77103124 22.49971999]
 

In [8]:
def lstm_step(x, prev_h, prev_c, Wx, Wh, b):
    
    N, H = prev_c.shape
    
    a = tf.matmul(x, Wx) + tf.matmul(prev_h, Wh) + b
    
    a_f = tf.math.sigmoid(a[:, 0:H])
    
    a_r = tf.math.sigmoid(a[:, H:2*H])
    
    a_s = tf.math.sigmoid(a[:, 2*H:3*H])
    
    a_g = tf.math.tanh(a[:, 3*H:4*H])
    
    next_c = a_f * prev_c + a_r * a_g
    
    next_h = a_s * tf.math.tanh(next_c)
    
    return next_h, next_c

In [9]:
def lstm_fwd(x, h_ini, c_ini, Wx, Wh, b):
    
    if x.ndim is not 3:
        x = tf.expand_dims(x, 0)
    
    t = x.shape[1]
    h = np.copy(h_ini)
    c = np.copy(c_ini)
    for i in range(t):
        xs = x[:, i, :]
        if xs.ndim is not 2:
            xs = tf.expand_dims(xs, 0)
        h, c = lstm_step(xs, h, c, Wx, Wh, b)
    return h

In [11]:
def dense(x, W, b):
    return tf.matmul(x, W) + b

In [12]:
state_size = 4

hl_ini = [[0.]*state_size]
c_ini = [[0.]*state_size]

Wx = tf.Variable(np.random.randn(num_features, state_size*4)*0.1)
Wh = tf.Variable(np.random.randn(state_size, state_size*4)*0.1)
b = tf.Variable(np.random.randn(state_size*4)*0.1)

Wd = tf.Variable(np.random.randn(state_size, out_size)*0.1)
bd = tf.Variable(np.random.randn(out_size)*0.1)

print(Wx, Wh)

<tf.Variable 'Variable:0' shape=(2, 16) dtype=float64, numpy=
array([[-0.0179505 , -0.0456883 ,  0.03207845, -0.00350919,  0.11529528,
        -0.0831654 ,  0.02231341,  0.00766288, -0.08454858, -0.08205871,
         0.07236798, -0.08369838,  0.05452022,  0.01436621,  0.18295203,
        -0.19371323],
       [-0.09928285, -0.05749889, -0.02100566, -0.03701541,  0.05499971,
         0.16422452, -0.06478951,  0.08458764, -0.08421951, -0.16151821,
         0.03038669, -0.18491711, -0.01923452, -0.12581617, -0.04603581,
        -0.08954482]])> <tf.Variable 'Variable:0' shape=(4, 16) dtype=float64, numpy=
array([[ 0.01106962, -0.0892606 , -0.15978592, -0.07802342, -0.05522391,
        -0.04921418, -0.05005516, -0.04075116,  0.08185176, -0.11827583,
        -0.04517619, -0.0790738 , -0.01076518, -0.01719981,  0.0244723 ,
        -0.10020346],
       [-0.11679509,  0.0813782 ,  0.1115818 , -0.03321786, -0.16517676,
        -0.08288083, -0.1359995 ,  0.18109243,  0.01130239,  0.11048858,
     

In [13]:
X, Y = seperate_labels(train_data, window=10, step=1)
x_cv, y_cv = seperate_labels(cv_data, 10, 1)

In [15]:
learning_rate = 0.1
mini_batch = 64
epochs = 100
for i in range(epochs):
    print("EPOCH", i)
    start = time.time()
    total_loss = 0.
    for j in range(0, num_samples-mini_batch, mini_batch):
        xs = X[j:j+mini_batch,:,:]
        ys = Y[j:j+mini_batch]
        with tf.GradientTape(persistent=True, watch_accessed_variables=False) as t:
            t.watch([Wx, Wh, b, Wd, bd])
            out = dense(lstm_fwd(xs, hl_ini, c_ini, Wx, Wh, b), Wd, bd)
            loss = tf.reduce_sum(tf.math.square(out-ys))/mini_batch
            total_loss += loss
        Wx.assign_sub(learning_rate*t.gradient(loss,Wx))
        Wh.assign_sub(learning_rate*t.gradient(loss,Wh))
        b.assign_sub(learning_rate*t.gradient(loss,b))
        Wd.assign_sub(learning_rate*t.gradient(loss,Wd))
        bd.assign_sub(learning_rate*t.gradient(loss,bd))
        del t
    end = time.time()
    mean_loss = total_loss/(num_samples//mini_batch)
    print("loss at end of EPOCH: %0.4f" % mean_loss)
    print("time taken: {:0.2f}s".format(end-start))
    y_predict = dense(lstm_fwd(x_cv, hl_ini, c_ini, Wx, Wh, b), Wd, bd)
    cv_loss = tf.reduce_sum(tf.math.square(y_predict-y_cv))/len(x_cv)
    print("validation loss:", cv_loss.numpy())
    print("Done with EPOCH")

EPOCH 0
loss at end of EPOCH: 0.0218
time taken: 0.99s
validation loss: 0.018230372330047276
Done with EPOCH
EPOCH 1
loss at end of EPOCH: 0.0218
time taken: 1.11s
validation loss: 0.018162610505896265
Done with EPOCH
EPOCH 2
loss at end of EPOCH: 0.0217
time taken: 0.95s
validation loss: 0.018105535900790212
Done with EPOCH
EPOCH 3
loss at end of EPOCH: 0.0217
time taken: 0.91s
validation loss: 0.01805799946096978
Done with EPOCH
EPOCH 4
loss at end of EPOCH: 0.0216
time taken: 0.92s
validation loss: 0.018018906413811236
Done with EPOCH
EPOCH 5
loss at end of EPOCH: 0.0216
time taken: 1.24s
validation loss: 0.017987231337505443
Done with EPOCH
EPOCH 6
loss at end of EPOCH: 0.0215
time taken: 0.99s
validation loss: 0.01796202710943814
Done with EPOCH
EPOCH 7
loss at end of EPOCH: 0.0215
time taken: 1.31s
validation loss: 0.017942429110444094
Done with EPOCH
EPOCH 8
loss at end of EPOCH: 0.0215
time taken: 1.14s
validation loss: 0.017927655876347242
Done with EPOCH
EPOCH 9
loss at end o

KeyboardInterrupt: 

In [19]:
test_data = data[1000:1150]
test_data,_,_ = normalize(test_data, train_mean, train_std)
x_test, y_test = seperate_labels(test_data, 10, 1)
print(x_test.shape, y_test.shape)
y_predict = dense(lstm_fwd(x_test, hl_ini, c_ini, Wx, Wh, b), Wd, bd)

print(y_predict.shape)
loss = tf.reduce_sum(tf.math.square(y_predict-y_test))/len(x_test)
print("Test error: ", loss.numpy())
#y_test = inverse_transform(y_test, train_mean, train_std)
#y_predict = inverse_transform(y_predict, train_mean, train_std)
final = np.hstack([y_test, y_predict])
print(final)
loss = tf.math.sqrt(tf.reduce_sum(tf.math.square(y_predict-y_test))/len(x_test))
print("Test error: ", loss.numpy())

(140, 9, 2) (140, 2)
(140, 2)
Test error:  0.019143371550185456
[[1.26259615 1.31703947 1.18350715 1.11102218]
 [1.22413462 1.52756579 1.19474595 1.11855156]
 [1.22413462 1.42230263 1.19620398 1.12077009]
 [1.30105769 1.15914474 1.20237436 1.12795949]
 [1.26259615 1.21177632 1.2001698  1.12748845]
 [1.22413462 1.26440789 1.20058549 1.12444641]
 [1.30105769 1.26440789 1.19620472 1.12122675]
 [1.18567308 1.15914474 1.19020571 1.11850721]
 [1.26259615 1.21177632 1.1995178  1.1230714 ]
 [1.26259615 1.31703947 1.17744942 1.11033739]
 [0.99336538 1.05388158 1.1824145  1.11260125]
 [1.18567308 1.00125    1.18904702 1.11710804]
 [1.26259615 1.15914474 1.12713315 1.08243376]
 [1.30105769 1.21177632 1.12610653 1.07886265]
 [1.26259615 1.15914474 1.14522347 1.08873029]
 [1.22413462 1.15914474 1.16479662 1.09951348]
 [1.07028846 1.05388158 1.16650077 1.10036074]
 [1.18567308 1.05388158 1.16161967 1.09809008]
 [1.14721154 1.15914474 1.12450681 1.07783845]
 [1.22413462 1.31703947 1.12484162 1.076812